In [1]:
import os
import tweepy
import pandas as pd
from dotenv import load_dotenv, find_dotenv

env_path = find_dotenv()
load_dotenv(env_path)

api_key = os.getenv('API_KEY')
api_key_secret = os.getenv('API_KEY_SECRET')
bearer_token = os.getenv('BEARER_TOKEN')

In [2]:
client = tweepy.Client(bearer_token=bearer_token)
auth = tweepy.OAuthHandler(api_key, api_key_secret)
api = tweepy.API(auth)

In [3]:
query = '(feliz OR felizmente) lang:pt'
#['word', '"words"', 'word1 OR word2, '@mention', '#hashtag', 'to:account', 'since:year-month-day']
# queries can be 512 characters long

In [4]:
tweets = client.search_recent_tweets(query=query, 
                                     tweet_fields=['author_id','created_at','source','entities'],
                                     max_results=10)

In [5]:
for tweet in tweets.data:
    print(tweet.text)
    print(f"Tweet id : {tweet.id}")
    print(f"Author id : {tweet.author_id}")
    print(f"Date : {tweet.created_at}")
    print(f"Source : {tweet.source}")
    if tweet.entities != None :
        if 'hashtags' in tweet.entities.keys():
            for i in range(len(tweet.entities['hashtags'])):
                hashtags = []
                hashtags.append(tweet.entities['hashtags'][i]['tag'])
            print(f"Hashtags : {hashtags}")
    print('------')

RT @VascodaGama: Hoje é dia de comemorar o aniversário do nosso zagueiro! 🥳

Feliz Cumple, Quintero! 🎉🎂

📸: Rafael Ribeiro | #VascoDaGama h…
Tweet id : 1506729339966181384
Author id : 4602622767
Date : 2022-03-23 20:27:22+00:00
Source : Twitter for Android
Hashtags : ['VascoDaGama']
------
@LariiPomper @delgallo_pedro Carai!! Isso que aconteceu com você não foi nem "falta de coversa", foi falta de caráter mesmo. Lamento por tudo isso e espero que você tenha conseguido superar, seguir feliz e que esteja com alguém que exalte todos os dias suas qualidades e faça você se sentir segura 🙂 Força!!
Tweet id : 1506729339483738114
Author id : 1498397274841485316
Date : 2022-03-23 20:27:22+00:00
Source : Twitter for Android
------
Qualquer coisa que te faz feliz vai inevitavelmente acabar, e quanto mais feliz você estiver, mais vai doer quando acabar…
Tweet id : 1506729337160355844
Author id : 1411819215795986439
Date : 2022-03-23 20:27:22+00:00
Source : Twitter for iPhone
------
O meu corpinho 

In [6]:
def hashtag_list(tweet):
    hashtags = []
    if tweet.entities != None :
        if 'hashtags' in tweet.entities.keys():
            for i in range(len(tweet.entities['hashtags'])):
                hashtags.append(tweet.entities['hashtags'][i]['tag'])
    return hashtags

In [7]:
tweets_search = [[],[],[],[],[]]
for tweet in tweets.data :
    tweets_search[0].append(tweet.text)
    tweets_search[1].append(tweet.author_id)
    tweets_search[2].append(tweet.created_at)
    tweets_search[3].append(tweet.source)
    tweets_search[4].append(hashtag_list(tweet))

In [8]:
tweets_search

[['RT @VascodaGama: Hoje é dia de comemorar o aniversário do nosso zagueiro! 🥳\n\nFeliz Cumple, Quintero! 🎉🎂\n\n📸: Rafael Ribeiro | #VascoDaGama h…',
  '@LariiPomper @delgallo_pedro Carai!! Isso que aconteceu com você não foi nem "falta de coversa", foi falta de caráter mesmo. Lamento por tudo isso e espero que você tenha conseguido superar, seguir feliz e que esteja com alguém que exalte todos os dias suas qualidades e faça você se sentir segura 🙂 Força!!',
  'Qualquer coisa que te faz feliz vai inevitavelmente acabar, e quanto mais feliz você estiver, mais vai doer quando acabar…',
  'O meu corpinho só precisa de tatuagem nova para eu ser feliz.',
  'RT @matheusl75: Se algum dia alguém falar mal do Beco da lama… feliz essa pessoa n é, pqp melhor rolê de Natal, onde q vc encontra 4 rodas…',
  'RT @_Kailanezinha: eu levo muito a sério o fato de quem não me da feliz aniversário',
  '@Silva_DG Feliz por isso, amo meus Zoeirões da Galáxia ❤️, estou sentindo falta do DG líder novamente, to

In [10]:
from nltk.tokenize import word_tokenize
import re

# define a preproc function
def preproc_func(tweet):
    '''Does the preprocessing of the tweets'''

    # stopwords: remove articles, prepositions, conjunctions etc
    stopwords=['rt','a','ah','g','h', 'cá','te','tu','tua','tuas','tém','um','uma','você','vocês','vos',
               'à','às','ao','aos','aquela','aquelas','aquele','aqueles','aquilo','as','até','com',
               'como','da','das','de','dela','delas','dele','deles','depois','do','dos','e','ela','elas',
               'ele','eles','em','entre','essa','essas','esse','esses','esta','eu','foi','fomos','for',
               'fora','foram','forem','formos','fosse','fossem','fui','fôramos','fôssemos', 'isso','isto',
               'já','lhe','lhes','me','mesmo','meu','meus','minha','minhas','muito','na','nas','no','nos',
               'nossa','nossas','nosso','nossos','num','numa','nós','oh','o','os','para','pela','pelas','pelo',
               'pelos','por','qual','quando','que','quem','se','seja','sejam','sejamos','sem','serei','seremos',
               'seria','seriam','será','serão','seríamos','seu','seus','somos','sou','sua','suas','são',
               'só','também','ah','q','g','oh','eh','vc','tbm','também','tambem','voceh','você','voce']

    tweet = tweet.lower() # lowercase

    tweet=re.sub('https?://[A-Za-z0-9./]+','',tweet) # remove links que começam com https?://
    tweet=re.sub('https://[A-Za-z0-9./]+','',tweet) # remove links que começam com https://
    tweet=re.sub('http://[A-Za-z0-9./]+','',tweet) # remove links que começam com http://

    tweet = re.sub(r'@[A-Za-z0-9_]+','',tweet) # remove @mentions
    tweet = re.sub(r'#','',tweet) # remove #hashtags

    tweet = re.sub(r'[^\w\s]','',tweet) # remove remove punctuation
    tweet = re.sub(r'[0-9]','',tweet) # remove numbers

    word_tokens=word_tokenize(tweet) # tokenize

    filtered_tweet = [w for w in word_tokens if not w in stopwords] # remove stopwords

    return filtered_tweet

In [11]:
preproc_tweets=[]
for tweet in tweets_search[0]:
    preproc_tweets.append(preproc_func(tweet))

In [13]:
preproc_tweets[0]

['saudade',
 'é',
 'enorme',
 'rebanho',
 'mas',
 'tempo',
 'feliz',
 'orgulhosa',
 'ver',
 'marilinha',
 'jamais',
 'esquecida']

In [ ]:
#for tweet in tweepy.Paginator(client.search_recent_tweets, query=query,
#                              tweet_fields=['context_annotations', 'created_at'], max_results=100).flatten(limit=1000):
#    print(tweet)

In [11]:
import stanza
stanza.download('pt')
nlp = stanza.Pipeline('pt')
def lemmatization(tweet):
    lemma = ""
    for sent in nlp(tweet).sentences:
        for word in sent.words:
            lemma += word.lemma + " "
    return lemma

2022-03-23 17:28:32 INFO: Downloading default packages for language: pt (Portuguese)...
2022-03-23 17:28:33 INFO: File exists: /Users/laurannefossat/stanza_resources/pt/default.zip.
2022-03-23 17:28:34 INFO: Finished downloading models and saved to /Users/laurannefossat/stanza_resources.
2022-03-23 17:28:34 INFO: Loading these models for language: pt (Portuguese):
| Processor | Package |
-----------------------
| tokenize  | bosque  |
| mwt       | bosque  |
| pos       | bosque  |
| lemma     | bosque  |
| depparse  | bosque  |

2022-03-23 17:28:34 INFO: Use device: cpu
2022-03-23 17:28:34 INFO: Loading: tokenize
2022-03-23 17:28:34 INFO: Loading: mwt
2022-03-23 17:28:34 INFO: Loading: pos
2022-03-23 17:28:34 INFO: Loading: lemma
2022-03-23 17:28:34 INFO: Loading: depparse
2022-03-23 17:28:34 INFO: Done loading processors!


In [13]:
%%time
preproc_tweets=[]
for tweet in tweets_search[0]:
    preproc_tweets.append(lemmatization(tweet))

CPU times: user 1.39 s, sys: 27.8 ms, total: 1.42 s
Wall time: 1.41 s


In [14]:
preproc_tweets

['RT @ VascodaGama : hoje ser dia de comemorar o aniversário de o nosso zagueiro ! 🥳 feliz Cumple , Quintero ! 🎉 🎂 ser : Rafael Ribeiro | # VascoDaGama h ',
 '@lariipomper @ delgallo _pedro Carai ! ! isso que acontecer com você não ser nem « falta de coversa « , ser falta de caráter mesmo . lamentar por tudo isso e esperar que você ter conseguir superar , seguir feliz e que estar com alguém que exaltar todo o dia seu qualidade e fazer você se sentir seguro ser Força ! ! ',
 'qualquer coisa que tu fazer feliz ir inevitavelmente acabar , e quanto mais feliz você estar , mais ir doer quando acabar ... ',
 'o meu corpinho só precisar de tatuagem novo para eu ser feliz . ',
 'RT @matheusl75 : se algum dia alguém falar mal de o Beco de o lama ... feliz esse pessoa n ser , pqp bom rolê de Natal , onde q vc encontrar 4 roda ... ',
 'RT @ _Kailanezinha : eu levar muito a sério o fato de quem não eu de o feliz aniversário ',
 '@silva _DG feliz por isso , amar meu Zoeirões de o Galáxia ❤️ , estar